In [ ]:
from __future__ import absolute_import

from perses.samplers import multistate
from perses.utils.openeye import *
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import LambdaProtocol
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import mdtraj as md
from openmmtools.constants import kB
from pkg_resources import resource_filename
from perses.tests.utils import validate_endstate_energies

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [2]:
from pkg_resources import resource_filename
from perses.app.relative_point_mutation_setup import PointMutationExecutor

# Sub class PointMutationExecutor for apo

In [13]:
class ApoPointMutationExecutor(PointMutationExecutor):
    def __init__(self,
                 receptor_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue, phase = 'complex',
                 conduct_endstate_validation = False,
                 forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 **kwargs):
        """
        arguments
            receptor_filename : str
                path to receptor; .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the hybrid topology factory
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.NoCutoff, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
        TODO : allow argument for separate apo structure if it exists separately
               allow argument for specator ligands besides the 'ligand_filename'
        """
        from openforcefield.topology import Molecule
        from openmmforcefields.generators import SystemGenerator

        # First thing to do is make a complex and apo...
        pdbfile = open(receptor_filename, 'r')
        pdb = app.PDBFile(pdbfile)
        pdbfile.close()
        receptor_positions, receptor_topology, receptor_md_topology = pdb.positions, pdb.topology, md.Topology.from_openmm(pdb.topology)
        receptor_topology = receptor_md_topology.to_openmm()
        receptor_n_atoms = receptor_md_topology.n_atoms

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields = forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                cache=None)

        # Create apo inputs...
        apo_topology, apo_positions, apo_system = self._solvate(receptor_topology, receptor_positions, 'tip3p', phase = phase)

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        point_mutation_engine = PointMutationEngine(wildtype_topology = apo_topology,
                                                             system_generator = self.system_generator,
                                                             chain_id = mutation_chain_id, #denote the chain id allowed to mutate (it's always a string variable)
                                                             max_point_mutants = 1,
                                                             residues_allowed_to_mutate = [mutation_residue_id], #the residue ids allowed to mutate
                                                             allowed_mutations = [(mutation_residue_id, proposed_residue)], #the residue ids allowed to mutate with the three-letter code allowed to change
                                                             aggregate = True) #always allow aggregation

        topology_proposal = point_mutation_engine.propose(apo_system, apo_topology)

        new_positions, logp_proposal = geometry_engine.propose(topology_proposal, apo_positions, beta)
        logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, apo_positions, beta)

        forward_htf = HybridTopologyFactory(topology_proposal = topology_proposal,
                                             current_positions =  apo_positions,
                                             new_positions = new_positions,
                                             use_dispersion_correction = False,
                                             functions=None,
                                             softcore_alpha = None,
                                             bond_softening_constant=1.0,
                                             angle_softening_constant=1.0,
                                             soften_only_new = False,
                                             neglected_new_angle_terms = [],
                                             neglected_old_angle_terms = [],
                                             softcore_LJ_v2 = True,
                                             softcore_electrostatics = True,
                                             softcore_LJ_v2_alpha = 0.85,
                                             softcore_electrostatics_alpha = 0.3,
                                             softcore_sigma_Q = 1.0,
                                             interpolate_old_and_new_14s = False,
                                             omitted_terms = None)

        if not topology_proposal.unique_new_atoms:
            assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
            assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            vacuum_added_valence_energy = 0.0
        else:
            added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

        if not topology_proposal.unique_old_atoms:
            assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
            assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
            subtracted_valence_energy = 0.0
        else:
            subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


        if conduct_endstate_validation:
            zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta = beta, ENERGY_THRESHOLD = ENERGY_THRESHOLD)
        else:
            pass

        self.apo_htf = forward_htf

In [17]:
apm_delivery = ApoPointMutationExecutor('../input/ala_vacuum.pdb', 
                         '1', 
                         '2', 
                         'VAL',
                        )
apo_htf = apm_delivery.get_apo_htf()

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[4, 4, 4, 4, 4, 4]
INFO:proposal_generator:Maximum atom matched after hydrogen exceptions: 2
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [18, 14, 16, 19, 21, 20, 17, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 48 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra tors

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.509035099709229.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.08293842667589812.
INFO:geometry:	reduced angle potential = 0.6194253769438156.
INFO:geometry:	reduced angle potential = 0.030346637540318577.
INFO:geometry:	reduced angle potential = 1.068531219068106.
INFO:geometry:	reduced angle potential = 1.8926670905584193.
INFO:geometry:	reduced angle potential = 0.0008237038458572306.
INFO:geometry:	reduced angle potential = 0.16255297674230146.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 48 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion

...

INFO:geometry:	reduced angle potential = 8.578546222633222e-06.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.82092023325904


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.5196339774707472), ('CustomTorsionForce', 0.0036836193671578284), ('CustomBondForce', 7.60944778229808)]


INFO:geometry:total reduced energy added from growth system: 9.132765379135986
INFO:geometry:final reduced energy 25.953685947680075
INFO:geometry:sum of energies: 25.953685612395027
INFO:geometry:magnitude of difference in the energies: 3.3528504772561973e-07
INFO:geometry:Final logp_proposal: 10.491261016710139
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getD

In [ ]:
# Now we can build the hybrid repex samplers
from perses.annihilation.lambda_protocol import LambdaProtocol
from openmmtools.multistate import MultiStateReporter
from perses.samplers.multistate import HybridRepexSampler
from openmmtools import mcmc

suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000


lambda_protocol = LambdaProtocol(functions='default')
reporter_file = 'reporter.nc'
reporter = MultiStateReporter(reporter_file, analysis_particle_indices = apo_htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                      collision_rate=5.0 / unit.picosecond,
                                                                      n_steps=250,
                                                                      reassign_velocities=False,
                                                                      n_restart_attempts=20,
                                                                      splitting="V R R R O R R R V",
                                                                      constraint_tolerance=1e-06),
                                                                      hybrid_factory=apo_htf, online_analysis_interval=10)
hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file = reporter, lambda_protocol = lambda_protocol, endstates=False)
hss.extend(n_cycles)

/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/openmmtools/multistate/multistatereporter.py:116: UserWarning: Warning: The openmmtools.multistate API is experimental and may change in future releases
  warnings.warn('Warning: The openmmtools.multistate API is experimental and may change in future releases')
/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/openmmtools/multistate/multistatesampler.py:145: UserWarning: Warning: The openmmtools.multistate API is experimental and may change in future releases
  warnings.warn('Warning: The openmmtools.multistate API is experimental and may change in future releases')
INFO:multistate:n_replicas not defined, setting to match n_states, 11


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

# Test revised PointMutationExecutor

### Solvent

In [1]:
from pkg_resources import resource_filename
from perses.app.relative_point_mutation_setup import PointMutationExecutor

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSC

DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$b68.31']
DEBUG:numba.core.byteflow:dispatch pc=72, inst=BINARY_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$b68.31', '$const70.32']
DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba

DEBUG:numba.core.interpreter:label 0:
    a = arg(0, name=a)                       ['a']
    b = arg(1, name=b)                       ['b']
    $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/numpy/__init__.py'>) ['$2load_global.0']
    $4load_method.1 = getattr(value=$2load_global.0, attr=zeros) ['$2load_global.0', '$4load_method.1']
    $const6.2 = const(int, 3)                ['$const6.2']
    $8call_method.3 = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None) ['$4load_method.1', '$8call_method.3', '$const6.2']
    c = $8call_method.3                      ['$8call_method.3', 'c']
    $const14.5 = const(int, 1)               ['$const14.5']
    $16binary_subscr.6 = getitem(value=a, index=$const14.5) ['$16binary_subscr.6', '$const14.5', 'a']
    $const20.8 = const(int, 2)               ['$const20.8']
    $22binary_subscr.9 = getitem(value=b, ind

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b4cfdcb7518>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: $8call_method.3 = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: c = $8call_method.3
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_get

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=14)
           2	LOAD_GLOBAL(arg=0, lineno=14)
           4	LOAD_METHOD(arg=1, lineno=14)
           6	LOAD_FAST(arg=0, lineno=14)
           8	LOAD_FAST(arg=0, lineno=14)
          10	CALL_METHOD(arg=2, lineno=14)
          12	STORE_FAST(arg=1, lineno=14)
          14	LOAD_GLOBAL(arg=0, lineno=15)
          16	LOAD_METHOD(arg=2, lineno=15)
          18	LOAD_FAST(arg=1, lineno=15)
          20	CALL_METHOD(arg=1, lineno=15)
          22	RETURN_VALUE(arg=None, lineno=15)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.cor

...

DEBUG:numba.core.ssa:on stmt: $const168.26 = const(float, -1.0)
DEBUG:numba.core.ssa:on stmt: $170compare_op.27 = cos_phi < $const168.26
DEBUG:numba.core.ssa:on stmt: branch $170compare_op.27, 174, 180
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 174
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b4cfe4e8780>
DEBUG:numba.core.ssa:on stmt: $const174.0 = const(float, -1.0)
DEBUG:numba.core.ssa:on stmt: cos_phi.1 = $const174.0
DEBUG:numba.core.ssa:on stmt: jump 192
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 180
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b4cfe4e8780>
DEBUG:numba.core.ssa:on stmt: $const182.1 = const(float, 1.0)
DEBUG:numba.core.ssa:on stmt: $184compare_op.2 = cos_phi > $const182.1
DEBUG:numba.core.ssa:on stmt: branch $184compare_op.2, 188, 192
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 188
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b4cfe4e8780>
DEBUG:numba.core.ssa:on stmt

DEBUG:numba.core.ssa:================================================================================
/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/perses-0.6.0.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:129: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  cos_phi = np.dot(plane1, plane2) / (_norm(plane1)*_norm(plane2))
/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/perses-0.6.0.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:137: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  if np.dot(a, plane2) <= 0:
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=106)
           2	LOAD_FAST(arg=0, lineno=106)
           4	LOAD_FAST(arg=1, lineno=106)
           6	BINARY_SUBTRACT(arg=None, lineno=106)
           8	RETURN_VALUE(arg=None, lineno=106)
DEBUG:numba.core.byteflow

DEBUG:numba.core.ssa:label 0:
    angle_position_1 = arg(0, name=angle_position_1) ['angle_position_1']
    bond_position_2 = arg(1, name=bond_position_2) ['bond_position_2']
    $6binary_subtract.2 = angle_position_1 - bond_position_2 ['$6binary_subtract.2', 'angle_position_1', 'bond_position_2']
    $8return_value.3 = cast(value=$6binary_subtract.2) ['$6binary_subtract.2', '$8return_value.3']
    return $8return_value.3                  ['$8return_value.3']

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b4cfe3b5f28>
DEBUG:numba.core.ssa:on stmt: angle_position_1 = arg(0, name=angle_position_1)
DEBUG:numba.core.ssa:on stmt: bond_position_2 = arg(1, name=bond_position_2)
DEBUG:numba.core.ssa:on stmt: $6binary_subtract.2 = angle_position_1 - bond_position_2
DEBUG:numba.core.ssa:on stmt: $8return_value.

DEBUG:numba.core.byteflow:Used_phis: defaultdict(<class 'set'>, {State(pc_initial=0 nstack_initial=0): set()})
DEBUG:numba.core.byteflow:defmap: {}
DEBUG:numba.core.byteflow:phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$a_12.0'}), (4, {'res': '$_32call_function_12_14.1'}), (6, {'lhs': '$a_12.0', 'rhs': '$_32call_function_12_14.1', 'res': '$6binary_true_divide.2'}), (8, {'retval': '$6binary_true_divide.2', 'castval': '$8return_value.3'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    _32call_function_12_1 = arg(0, name=_32call_func

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=109)
           2	LOAD_FAST(arg=1, lineno=109)
           4	LOAD_FAST(arg=0, lineno=112)
           6	BINARY_TRUE_DIVIDE(arg=None, lineno=112)
           8	RETURN_VALUE(arg=None, lineno=112)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=109)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_FAST(arg=1, lineno=109)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_FAST(arg=0, lineno=112)
DEBUG:numba.core.byteflow:stack ['$c_12.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=BINARY_TRUE_DIVIDE(arg=None, lineno=112)
DEBUG:numba.core.byteflow:stack ['$c_12.0', '$_56call_function_22_14.1']
DEBUG:numba.core.by

In [2]:
apm_delivery = PointMutationExecutor('../input/ala_vacuum.pdb', 
                         '1', 
                         '2', 
                         'VAL',
                        conduct_endstate_validation=True,
                        )
apo_htf = apm_delivery.get_apo_htf()

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[4, 4, 4, 4, 4, 4]
INFO:proposal_generator:Maximum atom matched after hydrogen exceptions: 2
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [14, 18, 21, 20, 17, 16, 15, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 48 angles in re

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.0399254815817493.
INFO:geometry:	reduced angle potential = 0.3991689514927569.
INFO:geometry:	reduced angle potential = 0.028334349504930523.
INFO:geometry:	reduced angle potential = 0.6729740938105145.
INFO:geometry:	reduced angle potential = 0.7243231975184564.
INFO:geometry:	reduced angle potential = 2.349731348925296.
INFO:geometry:	reduced angle potential = 0.5445886282874882.
INFO:geometry:	reduced angle potential = 0.8293967197122432.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	th

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 9.132765379135986
INFO:geometry:final reduced energy 25.953685947680075
INFO:geometry:sum of energies: 25.953685612395027
INFO:geometry:magnitude of difference in the energies: 3.3528504772561973e-07
INFO:geometry:Final logp_proposal: 10.446313318626755


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.5196339774707472), ('CustomTorsionForce', 0.0036836193671578284), ('CustomBondForce', 7.60944778229808)]


INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.7648, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.7648), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions 

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
			HarmonicBondForce: 0.03336689198711389
			HarmonicAngleForce: 2.1347394429482724
			PeriodicTorsionForce: 16.176131477654224
			NonbondedForce: -1285.9024711046145
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 0.0333668919871139
			HarmonicBondForce: 3.9751261280909644
			CustomAngleForce: 0.6151054654774569
			HarmonicAngleForce: 13.133425959114717
			CustomTorsionForce: 0.0014372414311074064
			PeriodicTorsionForce: 21.9062598581807
			NonbondedForce: -1331.6467615296067
			CustomNonbondedForce: -13.119137718048535
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
conducting subsequent work with the following platform: CUDA
			CustomBondForce: 0.0333668919871139
			HarmonicBondForce: 3.9751261280909644
			CustomAngleForce: 0.6151054654774569
			H

### Vacuum

In [1]:
from pkg_resources import resource_filename
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import simtk.openmm.app as app

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSC

DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$b68.31']
DEBUG:numba.core.byteflow:dispatch pc=72, inst=BINARY_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$b68.31', '$const70.32']
DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba

DEBUG:numba.core.interpreter:label 0:
    a = arg(0, name=a)                       ['a']
    b = arg(1, name=b)                       ['b']
    $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/numpy/__init__.py'>) ['$2load_global.0']
    $4load_method.1 = getattr(value=$2load_global.0, attr=zeros) ['$2load_global.0', '$4load_method.1']
    $const6.2 = const(int, 3)                ['$const6.2']
    $8call_method.3 = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None) ['$4load_method.1', '$8call_method.3', '$const6.2']
    c = $8call_method.3                      ['$8call_method.3', 'c']
    $const14.5 = const(int, 1)               ['$const14.5']
    $16binary_subscr.6 = getitem(value=a, index=$const14.5) ['$16binary_subscr.6', '$const14.5', 'a']
    $const20.8 = const(int, 2)               ['$const20.8']
    $22binary_subscr.9 = getitem(value=b, ind

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b7cd16f4ef0>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: $8call_method.3 = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: c = $8call_method.3
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_get

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=14)
           2	LOAD_GLOBAL(arg=0, lineno=14)
           4	LOAD_METHOD(arg=1, lineno=14)
           6	LOAD_FAST(arg=0, lineno=14)
           8	LOAD_FAST(arg=0, lineno=14)
          10	CALL_METHOD(arg=2, lineno=14)
          12	STORE_FAST(arg=1, lineno=14)
          14	LOAD_GLOBAL(arg=0, lineno=15)
          16	LOAD_METHOD(arg=2, lineno=15)
          18	LOAD_FAST(arg=1, lineno=15)
          20	CALL_METHOD(arg=1, lineno=15)
          22	RETURN_VALUE(arg=None, lineno=15)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.cor

...

DEBUG:numba.core.ssa:==== SSA block rewrite pass on 174
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b7cd197ef28>
DEBUG:numba.core.ssa:on stmt: $const174.0 = const(float, -1.0)
DEBUG:numba.core.ssa:on stmt: cos_phi.1 = $const174.0
DEBUG:numba.core.ssa:on stmt: jump 192
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 180
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b7cd197ef28>
DEBUG:numba.core.ssa:on stmt: $const182.1 = const(float, 1.0)
DEBUG:numba.core.ssa:on stmt: $184compare_op.2 = cos_phi > $const182.1
DEBUG:numba.core.ssa:on stmt: branch $184compare_op.2, 188, 192
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 188
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b7cd197ef28>
DEBUG:numba.core.ssa:on stmt: $const188.0 = const(float, 1.0)
DEBUG:numba.core.ssa:on stmt: cos_phi.2 = $const188.0
DEBUG:numba.core.ssa:on stmt: jump 192
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 192
DEBUG:numba.core.ss

DEBUG:numba.core.ssa:================================================================================
/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/perses-0.6.0.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:129: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  cos_phi = np.dot(plane1, plane2) / (_norm(plane1)*_norm(plane2))
/home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/perses-0.6.0.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:137: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  if np.dot(a, plane2) <= 0:
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=106)
           2	LOAD_FAST(arg=0, lineno=106)
           4	LOAD_FAST(arg=1, lineno=106)
           6	BINARY_SUBTRACT(arg=None, lineno=106)
           8	RETURN_VALUE(arg=None, lineno=106)
DEBUG:numba.core.byteflow

DEBUG:numba.core.ssa:label 0:
    angle_position_1 = arg(0, name=angle_position_1) ['angle_position_1']
    bond_position_2 = arg(1, name=bond_position_2) ['bond_position_2']
    $6binary_subtract.2 = angle_position_1 - bond_position_2 ['$6binary_subtract.2', 'angle_position_1', 'bond_position_2']
    $8return_value.3 = cast(value=$6binary_subtract.2) ['$6binary_subtract.2', '$8return_value.3']
    return $8return_value.3                  ['$8return_value.3']

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b7cd18e3320>
DEBUG:numba.core.ssa:on stmt: angle_position_1 = arg(0, name=angle_position_1)
DEBUG:numba.core.ssa:on stmt: bond_position_2 = arg(1, name=bond_position_2)
DEBUG:numba.core.ssa:on stmt: $6binary_subtract.2 = angle_position_1 - bond_position_2
DEBUG:numba.core.ssa:on stmt: $8return_value.

DEBUG:numba.core.byteflow:Used_phis: defaultdict(<class 'set'>, {State(pc_initial=0 nstack_initial=0): set()})
DEBUG:numba.core.byteflow:defmap: {}
DEBUG:numba.core.byteflow:phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$a_12.0'}), (4, {'res': '$_32call_function_12_14.1'}), (6, {'lhs': '$a_12.0', 'rhs': '$_32call_function_12_14.1', 'res': '$6binary_true_divide.2'}), (8, {'retval': '$6binary_true_divide.2', 'castval': '$8return_value.3'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    _32call_function_12_1 = arg(0, name=_32call_func

DEBUG:numba.core.ssa:================================================================================
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=109)
           2	LOAD_FAST(arg=1, lineno=109)
           4	LOAD_FAST(arg=0, lineno=112)
           6	BINARY_TRUE_DIVIDE(arg=None, lineno=112)
           8	RETURN_VALUE(arg=None, lineno=112)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=109)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_FAST(arg=1, lineno=109)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_FAST(arg=0, lineno=112)
DEBUG:numba.core.byteflow:stack ['$c_12.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=BINARY_TRUE_DIVIDE(arg=None, lineno=112)
DEBUG:numba.core.byteflow:stack ['$c_12.0', '$_56call_function_22_14.1']
DEBUG:numba.core.by

In [8]:
apm_delivery = PointMutationExecutor('../input/ala_vacuum.pdb', 
                         '1', 
                         '2', 
                         'VAL',
                        phase='vacuum',
                        barostat=None,
                        periodic_forcefield_kwargs=None, 
                        nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}
                        )
apo_htf = apm_delivery.get_apo_htf()

2020-06-01 13:40:47,231:(94.40s):perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
2020-06-01 13:40:47,246:(0.01s):proposal_generator:	Conducting polymer point mutation proposal...
2020-06-01 13:40:47,257:(0.01s):proposal_generator:[4, 4, 4, 4, 4, 4]
2020-06-01 13:40:47,260:(0.00s):proposal_generator:	rank_degenerate_maps: 'NoneType' object has no attribute 'IsAromatic'
2020-06-01 13:40:47,263:(0.00s):proposal_generator:Maximum atom matched after hydrogen exceptions: 2
2020-06-01 13:40:47,281:(0.02s):geometry:propose: performing forward proposal
2020-06-01 13:40:47,284:(0.00s):geometry:propose: unique new atoms detected; proceeding to _logp_propose...
2020-06-01 13:40:47,286:(0.00s):geometry:Conducting forward proposal...
2020-06-01 13:40:47,289:(0.00s):geometry:Computing proposal order with NetworkX...
2020-06-01 13:40:47,294:(0.01s):geometry:number of atoms to be placed: 8
2020-06-01 13:40:47,297:(0.00s):geometry:Atom index proposal order is [14, 18,

conducting subsequent work with the following platform: CUDA


2020-06-01 13:40:48,023:(0.63s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


2020-06-01 13:40:48,563:(0.54s):geometry:There are 8 new atoms
2020-06-01 13:40:48,570:(0.01s):geometry:	reduced angle potential = 0.0407752800482258.
2020-06-01 13:40:48,696:(0.13s):geometry:	reduced angle potential = 0.04944835780520963.
2020-06-01 13:40:48,817:(0.12s):geometry:	reduced angle potential = 0.04677096785714798.
2020-06-01 13:40:48,937:(0.12s):geometry:	reduced angle potential = 0.1552786173074734.
2020-06-01 13:40:49,058:(0.12s):geometry:	reduced angle potential = 0.3529251674562445.
2020-06-01 13:40:49,178:(0.12s):geometry:	reduced angle potential = 1.8370270100270991.
2020-06-01 13:40:49,299:(0.12s):geometry:	reduced angle potential = 0.233568867494854.
2020-06-01 13:40:49,421:(0.12s):geometry:	reduced angle potential = 0.04567782929077883.
2020-06-01 13:40:49,543:(0.12s):geometry:	beginning construction of no_nonbonded final system...
2020-06-01 13:40:49,547:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'Period

...

2020-06-01 13:40:55,763:(1.34s):geometry:total reduced potential before atom placement: 16.82092023325904


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2020-06-01 13:40:57,436:(1.67s):geometry:total reduced energy added from growth system: 9.132765379135986
2020-06-01 13:40:57,439:(0.00s):geometry:final reduced energy 25.953685947680015
2020-06-01 13:40:57,442:(0.00s):geometry:sum of energies: 25.953685612395027
2020-06-01 13:40:57,444:(0.00s):geometry:magnitude of difference in the energies: 3.352849873294872e-07
2020-06-01 13:40:57,447:(0.00s):geometry:Final logp_proposal: 9.797851547049408


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.5196339774707472), ('CustomTorsionForce', 0.0036836193671578284), ('CustomBondForce', 7.60944778229808)]


2020-06-01 13:40:57,708:(0.26s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2020-06-01 13:40:57,720:(0.01s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2020-06-01 13:40:57,722:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2020-06-01 13:40:57,725:(0.00s):relative:No unknown forces.
2020-06-01 13:40:57,727:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 0
2020-06-01 13:40:57,730:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2020-06-01 13:40:57,732:(0.00s):relative:Adding and mapping new atoms to hybrid system...
2020-06-01 13:40:57,735:(0.00s):relative:No MonteCarloBarostat added.
2020-06-01 13:40:57,738:(0.00s):relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(va

In [9]:
import logging
import datetime

# Set up logger
class TimeFilter(logging.Filter):
    def filter(self, record):
        try:
            last = self.last
        except AttributeError:
            last = record.relativeCreated
        delta = datetime.datetime.fromtimestamp(record.relativeCreated/1000.0) - datetime.datetime.fromtimestamp(last/1000.0)
        record.relative = '{0:.2f}'.format(delta.seconds + delta.microseconds/1000000.0)
        self.last = record.relativeCreated
        return True
fmt = logging.Formatter(fmt="%(asctime)s:(%(relative)ss):%(name)s:%(message)s")
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S')
_logger = logging.getLogger()
_logger.setLevel(logging.DEBUG)
[hndl.addFilter(TimeFilter()) for hndl in _logger.handlers]
[hndl.setFormatter(fmt) for hndl in _logger.handlers]



[None]

In [10]:
# Now we can build the hybrid repex samplers
from perses.annihilation.lambda_protocol import LambdaProtocol
from openmmtools.multistate import MultiStateReporter
from perses.samplers.multistate import HybridRepexSampler
from openmmtools import mcmc
import simtk.unit as unit


suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000


lambda_protocol = LambdaProtocol(functions='default')
reporter_file = 'reporter.nc'
reporter = MultiStateReporter(reporter_file, analysis_particle_indices = apo_htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                      collision_rate=5.0 / unit.picosecond,
                                                                      n_steps=250,
                                                                      reassign_velocities=False,
                                                                      n_restart_attempts=20,
                                                                      splitting="V R R R O R R R V",
                                                                      constraint_tolerance=1e-06),
                                                                      hybrid_factory=apo_htf, online_analysis_interval=10)
hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file = reporter, lambda_protocol = lambda_protocol, endstates=False)
hss.extend(n_cycles)

2020-06-01 13:41:32,979:(0.00s):openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as reporter_checkpoint.nc
2020-06-01 13:41:33,033:(0.05s):multistate:n_replicas not defined, setting to match n_states, 11
2020-06-01 13:41:43,071:(10.04s):mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
2020-06-01 13:41:43,075:(0.00s):mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b7cd51367b8>>
2020-06-01 13:41:43,083:(0.01s):mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b7cc4f14d08>
2020-06-01 13:41:43,085:(0.00s):mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b7cc4ed1268>
2020-06-01 13:41:43,088:(0.00s):mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b7cc4ed1510>
2020-06-01 13:41:43,126:(0.04s):openm

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

2020-06-01 13:41:43,306:(0.09s):mpiplus.mpiplus:Single node: executing <bound method MultiStateSampler._store_options of <instance of HybridRepexSampler>>
2020-06-01 13:41:43,308:(0.00s):openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
2020-06-01 13:41:43,313:(0.00s):mpiplus.mpiplus:Running _compute_replica_energies serially.
2020-06-01 13:41:43,813:(0.50s):openmmtools.utils:Computing energy matrix took    0.501s
2020-06-01 13:41:43,817:(0.00s):mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b7cd51367b8>>
2020-06-01 13:41:43,821:(0.00s):openmmtools.multistate.multistatesampler:********************************************************************************
2020-06-01 13:41:43,824:(0.00s):openmmtools.multistate.multistatesampler:Iteration 1/5000
2020-06-01 13:41:43,827:(0.00s):openmmtools.multistate.multistatesampler:********************

2020-06-01 13:41:48,198:(0.00s):openmmtools.multistate.multistatesampler:Propagating all replicas...
2020-06-01 13:41:48,199:(0.00s):mpiplus.mpiplus:Running _propagate_replica serially.
2020-06-01 13:41:49,281:(1.08s):mpiplus.mpiplus:Running _get_replica_move_statistics serially.
2020-06-01 13:41:49,285:(0.00s):openmmtools.utils:Propagating all replicas took    1.087s
2020-06-01 13:41:49,288:(0.00s):mpiplus.mpiplus:Running _compute_replica_energies serially.
2020-06-01 13:41:49,389:(0.10s):openmmtools.utils:Computing energy matrix took    0.101s
2020-06-01 13:41:49,392:(0.00s):mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b7cc4ed1620>
2020-06-01 13:41:49,394:(0.00s):mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b7cc4ed18c8>
2020-06-01 13:41:49,397:(0.00s):openmmtools.multistate.multistatereporter:Iteration 3 not on the Checkpoint Interval of 10. Sampler State not written.
2020-06-01 13:41:49

KeyboardInterrupt: 